In [131]:
import pandas as pd
df = pd.read_pickle('trains.pkl')

df = df[df['duration'] >= 0]

In [154]:
import matplotlib.pyplot as plt
%matplotlib inline
enter_count = df.start.value_counts()
exit_count = df.end.value_counts()
df2 = pd.concat([enter_count, exit_count], axis=1, keys=["enter", "exit"])

df2.fillna(0, inplace=True)

df2["train"] = df2["enter"] - df2["exit"]
print(df2.head())
counts = df2["train"].resample("1min").sum().fillna(0)
print(counts.head(50))

                     enter  exit  train
2017-10-09 00:06:23    1.0   0.0    1.0
2017-10-09 00:11:14    0.0   1.0   -1.0
2017-10-09 00:38:23    1.0   0.0    1.0
2017-10-09 00:39:34    0.0   1.0   -1.0
2017-10-09 00:41:01    1.0   0.0    1.0
2017-10-09 00:06:00    1.0
2017-10-09 00:07:00    0.0
2017-10-09 00:08:00    0.0
2017-10-09 00:09:00    0.0
2017-10-09 00:10:00    0.0
2017-10-09 00:11:00   -1.0
2017-10-09 00:12:00    0.0
2017-10-09 00:13:00    0.0
2017-10-09 00:14:00    0.0
2017-10-09 00:15:00    0.0
2017-10-09 00:16:00    0.0
2017-10-09 00:17:00    0.0
2017-10-09 00:18:00    0.0
2017-10-09 00:19:00    0.0
2017-10-09 00:20:00    0.0
2017-10-09 00:21:00    0.0
2017-10-09 00:22:00    0.0
2017-10-09 00:23:00    0.0
2017-10-09 00:24:00    0.0
2017-10-09 00:25:00    0.0
2017-10-09 00:26:00    0.0
2017-10-09 00:27:00    0.0
2017-10-09 00:28:00    0.0
2017-10-09 00:29:00    0.0
2017-10-09 00:30:00    0.0
2017-10-09 00:31:00    0.0
2017-10-09 00:32:00    0.0
2017-10-09 00:33:00    0.0
2017

In [146]:
df3 = counts.to_frame()
df3['dow'] = df3.index.dayofweek
df3['minute'] = df3.index.minute + (60*df3.index.hour)
df3['hour'] = df3.index.hour
print(len(df3))
df3.tail()

38865


,train,dow,minute,hour
2017-11-04 23:46:00,0.0,5,1426,23
2017-11-04 23:47:00,0.0,5,1427,23
2017-11-04 23:48:00,1.0,5,1428,23
2017-11-04 23:49:00,1.0,5,1429,23
2017-11-04 23:50:00,0.0,5,1430,23


In [147]:
import numpy as np

df3['is_training'] = np.random.uniform(0, 1, len(df3)) <= .75
# Create two new dataframes, one with the training rows, one with the test rows
train, test = df3[df3['is_training']==True], df3[df3['is_training']==False]
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 29207
Number of observations in the test data: 9658


In [148]:
y = pd.factorize(train['train'])[0]
features = df3.columns[1:4]
print(features)

Index(['dow', 'minute', 'hour'], dtype='object')


In [149]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score


# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(train[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [150]:
predict=clf.predict(test[features])
accuracy_score(predict,test['train'])

0.22406295299233797

In [151]:
pd.crosstab(test['train'], predict, rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0.0,885,7319
1.0,175,1279


In [152]:
list(zip(train[features], clf.feature_importances_))

[('dow', 0.10396590195055908),
 ('minute', 0.86857157066408619),
 ('hour', 0.027462527385354835)]